

This coding script makes:

     1. Transform dicom files to nifti format:
         The code will transform dicom files to the nifti format 
         by protecting the hierarchy with input data folders
     2. Record acquisition informations to an excel file:
         The code will give an excel file which is including dicom information 
         together with corresponding assigned id of each case
     
----------

## Contents

-    [1. importing necessary libraries](#1)    
         1.1 importing puclic libraries:
               --if this box is giving an error, this mean you need to check libraries in this cell, 
               you need    to install them               
         1.2 importing user-defined libraries 
               --if this box is giving an error, this mean you need to check
               ipynb file(jupyter notebook) with the name of  'project_helper_functions_classes' 
               within your working directory
                              
-    [2. Preperation of input informations](#2)    
    
        -- This part for setting folder and file names 
        for correctly loading inputs and getting desired output paths
        -- only THIS PART COULD BE EDIT depend on your data storage structure.
        -- if you would like to change name of the folders or files, 
        please change only right side of equation carefully without changing the structure of the object
        
        IMPORTANT NOTE for storaging input data:
           -- The folders' hierarchy should be as follows:                       
                    *input_data_folder 
                        ** main_folders(e.g. : patients, controls) 
                           IMPORTANT NOTE: patients' folder name should begin with 'p' 
                           IMPORTANT NOTE: controls' folder name should begin with 'c' 
                            *** case_folders(e.g.:caseX) 
                                **** MRI_type_folder(e.g.: t2_coronal, t1_axial ) 
                                    ***** MRI_files(e.g. : mri_.dcm)                                    
           -- Please storage only related folders and files inside each folder. 
               Othervise, code may not work as desired
               
-    [3. Converting dicom files to nifti format](#3)        
          3.1 preparation for converting dicom files into nifti form         
          3.2 converting each dicom file to nitfi format with protecting hierarchy and folder names
          3.3 create simple hierarchy for using nifti files in further proccess
            -- New hierarchy of data storage will be as follows:    
                *ordered_nifti_folder
                    ** main_folders(e.g. : patient_folder, control_folder)
                        *** MRI_type_folder(e.g.: t2_tra, t1_axial ) 
                            **** nifti files(e.g. : case1.nii.gz)            
    
-    [4.  Acquire dicom files informations and update storage hierarchy by assigning specific numbers to cases](#4)   
           4.1 preparation for recording informations          
           4.2 store acquisition info from dcm files

-    [5. Save informations to excel file](#5)
    

<a name='1'></a>
# 1. importing necessary libraries

### 1.1 importing puclic libraries

In [ ]:
import os 
import xlsxwriter
import pandas as pd
import shutil

### 1.2 importing user-defined libraries

In [1]:
from ipynb.fs.full.project_helper_functions_classes import create_folder, convert_dcm_folders_to_nifti_folder,take_acquisition_info

<a name='2'></a>
# 2. Preperation of input informations 

In [ ]:
## spesify input data folder name and output data folder name 
 
# 'img_folder' should be exist before coding script and includes all types image data 
img_folder = 'data'   

# 'dcm_folder' should be exist before coding script and includes dicom images folders (this folder within img_folder)
dcm_folder = 'dicom_data'      

## 'nifti_folder' will be created with code,  and will include nifti images folders (this folder within img_folder)
nifti_folder = 'nifti' 

## 'output_folder' will be created with code, and will excel file will be saved into it
output_folder = 'output'         

## write 2 main folders for patients and controls which are including cases datasets(under dcm_folder) 
## note: there should be 2 main folders, first letter should be 'p' : for patient folder, 'c' : for controls
mainfolders = ["patients", "controls"]    

## write all MRI types you have (it is not important if some patients does not have each type data)
subfolder_list = ['t2_tra', 't2_cor', 't1_tra', 't1_sag', 't2_darkfluid_tra']

## chose a representative word for saving and using your data during analysis
case_sign = 'case'

## choose a name for excel file for recording acquisition informations
excel_name = 'MRI_informations.xlsx'


<a name='3'></a>
# 3. Converting dicom files to nifti format

### 3.1 preparation for converting dicom files into nifti form

In [ ]:
nifti_folder_path = create_folder(os.path.join(img_folder, nifti_folder)) 
dicom_folder_path = os.path.abspath(os.path.join(img_folder, dcm_folder))

### 3.2 converting each dicom file to nitfi format with protecting hierarchy and folder names

In [ ]:
dcm_converter = convert_dcm_folders_to_nifti_folder()
dcm_converter.transform_files(dicom_folder_path, nifti_folder_path)

### 3.3 create simple hierarchy for using nifti files in further proccess

In [ ]:
ordered_data_path = create_folder(os.path.join(img_folder , 'ordered_data')) 
for i, mainfolder in enumerate(mainfolders):
    ordered_main_folder_path = create_folder(os.path.join(ordered_data_path, mainfolder))    
    for subfolder in subfolder_list:        
        create_folder(os.path.join(ordered_main_folder_path, subfolder))    

<a name='4'></a>
# 4.  Acquire dicom files informations and update storage hierarchy by assigning specific numbers to cases

### 4.1 preparation for recording information

In [ ]:
columns = ['input_name', 'case_name','TR', 'TE', 'FA', 'Percent_Phase_FOV', 
           'Spacing_Between_Slices', 'Slice_Thickness'
        'Rows', 'Columns', 'Pixel_Spacing', 'Acquisition_Matrix', 
           'MR_Acquisition_Type', 'Manufacturer'
        'Magnetic_Field_Strength']

df_mri = {}
for mainfolder in mainfolders:
    df_mri[mainfolder] = {}
    for subfolder in subfolder_list:
        df_mri[mainfolder][subfolder] = pd.DataFrame(columns = columns)

### 4.2 store acquisition info from dcm files

In [ ]:
for mainfolder in mainfolders:    
    case_names = take_folder_list(os.path.join(dicom_folder_path, mainfolder))       
    for case in case_names:        
        for subfolder in subfolder_list:            
            nifti_subfolder_path = os.path.join(nifti_folder_path, mainfolder , case, subfolder)            
            if os.path.isdir(nifti_subfolder_path):                
                dicom_path = os.path.join(dicom_folder_path, mainfolder, case, subfolder)                               
                case_id = case_sign + str(len(df_mri[mainfolder][subfolder]) + 1)                 
                dicom_info = take_acquisition_info(dicom_path,  case, case_id)
                df_mri[mainfolder][subfolder] = pd.concat([df_mri[mainfolder][subfolder], dicom_info], 
                                              axis=0, join='outer', ignore_index=True)
                target = os.path.join(ordered_data_path, mainfolder, subfolder, case_id + '.nii.gz')                
                source_file = [file for file in os.listdir(nifti_subfolder_path) if '.nii.gz' in file][0]
                source_path = os.path.join(nifti_subfolder_path, source_file)                
                shutil.copy(source_path, target)

<a name='5'></a>
# 5. Save informations to excel file

In [ ]:
output_folder_path = create_folder(output_folder)
excel_path = os.path.join(output_folder_path, excel_name)
with pd.ExcelWriter(excel_path) as writer:  
    for mainfolder in mainfolders:
        for subfolder in subfolder_list:
            sheetname = mainfolder + '_' + subfolder            
            df_mri[mainfolder][subfolder].to_excel(writer, sheet_name = sheetname)        